In [1]:
import pandas  as pd
import numpy as  np
import selenium as sl
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import requests
import json

from pprint import pprint

In [2]:
driver = webdriver.Chrome("chromedriver.exe")

In [39]:
driver.get("https://www.stephaneplazaimmobilier.com/")

In [6]:
def fermer_popup():
    driver.find_element_by_xpath('//*[@id="home-popup"]/div/div[1]/button/i').click()

In [15]:
fermer_popup()

## create_link_ville

In [18]:
def check_exist(a,b):
    list_a=a.split(" ")
    return all(x in b for x in list_a)
def create_link_ville(ville):
    from time import sleep
    elem = driver.find_element_by_class_name("el-select__input")
    elem.send_keys(Keys.LEFT_SHIFT ,Keys.LEFT_CONTROL  ,Keys.LEFT )
    elem.send_keys(Keys.DELETE )
    elem.send_keys(ville)
    sleep(5)
    els = driver.find_elements_by_class_name('el-select-dropdown__item')
    sleep(5)
    for el in els:
        for option in el.find_elements_by_tag_name('span'):
            if check_exist(ville, re.findall(r'<span>(.*?)</span>',el.get_attribute('innerHTML'))[0]):
                sleep(5)
                option.click()
                
                break
    sleep(3)
    submit_button = driver.find_elements_by_xpath('//*[@id="search"]/div[1]/div[2]/form/div[3]/button')[0]
    submit_button.click()
    sleep(10)
    return str(driver.current_url)

In [80]:
def select_type(t):
    er=0
    for i in range(len(driver.find_elements_by_xpath('//*[@id="search"]//label'))):

        if driver.find_elements_by_xpath('//*[@id="search"]//label')[i].get_attribute('innerHTML').lower() == t.lower():
            driver.find_elements_by_xpath('//*[@id="search"]//label')[i].click()
            er=1
    if er==0 :print("parameter not found please type either 'LOUER' or 'ACHETER'") 


In [84]:
select_type("LOUER")
ou='Paris 12ème arrondissement'
link_ville=create_link_ville(ou)
link_ville

'https://www.stephaneplazaimmobilier.com/immobilier-acheter?target=rent&location=75112&now=1&page=1'

## create_links_annonces

In [85]:
def get_post_links(lin):
    driver.get(lin)
    from time import sleep
    sleep(8)
    a=1
    while a==1 :
        current_page=re.findall('\d+$',str(driver.current_url) )[0]
        try :
            
            element = driver.find_element_by_xpath("(//div[contains(@class, 'room purchase')])[last()]")
            element.location_once_scrolled_into_view
        except:
            print("error No data")
        sleep(5)
        last_page=re.findall('\d+$',str(driver.current_url) )[0]
        if (current_page == last_page):
            a=0
    sleep(2)
    links = []
    i=0
    for l in  driver.find_elements_by_xpath("(//a[contains(@class, 'title-wrap')])"):
        links.append(l.get_attribute('href'))
    return links

In [21]:
list_links=[]
list_links= get_post_links(link_ville)

## formater_annonce

In [33]:
def scraping_LINKS(l):
    from time import sleep
    df_list=[]
    j=0
    for i in l:
        j+=1
        sleep(5)
        driver.get(i)
        d=formater_annonce()
        df_list.append(d)
        print(str(j)+"//"+str(d))
    return df_list

In [34]:
def formater_annonce():
    # ID annonce
    id_annonce = re.findall('/\d+/',str(driver.current_url) )[0].replace('/','')
    element=driver.find_element_by_class_name("fiche-de-wrap")
    html_description=str(element.get_attribute('innerHTML'))
    # Type de Bien
    type_bein=re.findall('<label>Type de bien</label> <span class="value">(.*?)</span>',re.sub(r'\s\s+', ' ', html_description))[0].strip()
    # Type_bien_corrige
    Type_bien_corrige= type_bein if type_bein=="Appartement" else type_bein[0].upper()
    # Surface
    surface=int(re.findall("^\d+",re.findall('<label>Surface</label> <span class="value">(.*?)</span>',re.sub(r'\s\s+', ' ', html_description))[0].strip())[0])
    # PRIX
    prix_bien=int(re.sub(r"\s+", "",re.findall("\d*\s*\d*",re.findall('<label>Prix de vente</label> <span class="value">(.*?)</span>',re.sub(r'\s\s+', ' ', html_description))[0].strip())[0] ))
    # Coordination
    ele=driver.find_element_by_xpath("(//div[@id='cityscan-widget'])")
    html_cityscan=str(ele.get_attribute('innerHTML'))
    json_url=re.sub(r"amp;", "",re.sub(r"%3A", ":",re.findall('src="(.*?)"',html_cityscan)[0].strip()))
    raw = requests.get(json_url).text
    raw=re.sub(r'\s+', '', raw)
    longitude=float(re.findall('"lon":(.*?),',raw)[0])
    lattitude=float(re.findall('"lat":(.*?),',raw)[0])
    return [id_annonce,type_bein,Type_bien_corrige,surface,longitude,lattitude,prix_bien]

In [36]:
list_=[]
list_=scraping_LINKS(list_links)

1//['2496889', 'Appartement', 'Appartement', 51, 2.40489, 48.842157, 537500]
2//['2496752', 'Appartement', 'Appartement', 43, 2.407693, 48.845402, 400000]
3//['1997588', 'Appartement', 'Appartement', 33, 2.377109, 48.841522, 349500]
4//['2293380', 'Appartement', 'Appartement', 24, 2.384823, 48.849557, 310000]
5//['1843928', 'Appartement', 'Appartement', 20, 2.403681, 48.841785, 224600]
6//['2293067', 'Appartement', 'Appartement', 43, 2.305336, 48.843353, 449900]
7//['2486727', 'Appartement', 'Appartement', 74, 2.396388, 48.832873, 780000]
8//['1715172', 'Appartement', 'Appartement', 28, 2.389965, 48.840425, 398000]
9//['2429690', 'Parking / box', 'P', 13, 2.393023, 48.842026, 25000]
10//['2370163', 'Appartement', 'Appartement', 77, 2.409114, 48.841521, 765000]
11//['2460586', 'Appartement', 'Appartement', 74, 2.396388, 48.832873, 780000]
12//['2336228', 'Appartement', 'Appartement', 15, 2.379975, 48.845908, 179500]
13//['2371974', 'Appartement', 'Appartement', 52, 1.999276, 48.829223, 

In [37]:
df = pd.DataFrame(list_,columns =["id_annonce","type_bein","Type_bien_corrige","surface","longitude","lattitude","prix_bien"])
df

,id_annonce,type_bein,Type_bien_corrige,surface,longitude,lattitude,prix_bien
0,2496889,Appartement,Appartement,51,2.404890,48.842157,537500
1,2496752,Appartement,Appartement,43,2.407693,48.845402,400000
2,1997588,Appartement,Appartement,33,2.377109,48.841522,349500
3,2293380,Appartement,Appartement,24,2.384823,48.849557,310000
4,1843928,Appartement,Appartement,20,2.403681,48.841785,224600
5,2293067,Appartement,Appartement,43,2.305336,48.843353,449900
6,2486727,Appartement,Appartement,74,2.396388,48.832873,780000
7,1715172,Appartement,Appartement,28,2.389965,48.840425,398000
8,2429690,Parking / box,P,13,2.393023,48.842026,25000
9,2370163,Appartement,Appartement,77,2.409114,48.841521,765000


## To JSON

In [38]:
out = df.to_json(orient='records')
out

'[{"id_annonce":"2496889","type_bein":"Appartement","Type_bien_corrige":"Appartement","surface":51,"longitude":2.40489,"lattitude":48.842157,"prix_bien":537500},{"id_annonce":"2496752","type_bein":"Appartement","Type_bien_corrige":"Appartement","surface":43,"longitude":2.407693,"lattitude":48.845402,"prix_bien":400000},{"id_annonce":"1997588","type_bein":"Appartement","Type_bien_corrige":"Appartement","surface":33,"longitude":2.377109,"lattitude":48.841522,"prix_bien":349500},{"id_annonce":"2293380","type_bein":"Appartement","Type_bien_corrige":"Appartement","surface":24,"longitude":2.384823,"lattitude":48.849557,"prix_bien":310000},{"id_annonce":"1843928","type_bein":"Appartement","Type_bien_corrige":"Appartement","surface":20,"longitude":2.403681,"lattitude":48.841785,"prix_bien":224600},{"id_annonce":"2293067","type_bein":"Appartement","Type_bien_corrige":"Appartement","surface":43,"longitude":2.305336,"lattitude":48.843353,"prix_bien":449900},{"id_annonce":"2486727","type_bein":"Ap

In [ ]:
with open('annonces .txt', 'w') as f:
    f.write(out)